In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np
import torch
import recipe_encoder as recipe

%load_ext autoreload
%autoreload 2

csv_file = 'archive/updated_data_with_lists.csv'
df = pd.read_csv(csv_file, converters={"Cleaned_Ingredients": literal_eval, "Instructions": literal_eval})
print(len(df))
df.head()

13496


,Unnamed: 0,Title,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[Pat chicken dry with paper towels, season all...",miso-butter-roast-chicken-acorn-squash-panzanella,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa..."
1,1,Crispy Salt and Pepper Potatoes,[Preheat oven to 400°F and line a rimmed bakin...,crispy-salt-and-pepper-potatoes-dan-kluger,"[2 large egg whites, 1 pound new potatoes (abo..."
2,2,Thanksgiving Mac and Cheese,[Place a rack in middle of oven; preheat to 40...,thanksgiving-mac-and-cheese-erick-williams,"[1 cup evaporated milk, 1 cup whole milk, 1 ts..."
3,3,Italian Sausage and Bread Stuffing,[Preheat oven to 350°F with rack in middle. Ge...,italian-sausage-and-bread-stuffing-240559,"[1 (¾- to 1-pound) round Italian loaf, cut int..."
4,4,Newton's Law,[Stir together brown sugar and hot water in a ...,newtons-law-apple-bourbon-cocktail,"[1 teaspoon dark brown sugar, 1 teaspoon hot w..."


In [2]:
column = df["Cleaned_Ingredients"]
# item_list = [word
#                     for w_list in column
#                     for str_list in w_list
#                     # Make period and comma separate words
#                     # # Remove parentheses and quotes
#                     for word in str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()]

item_list = []
element_list = []
ingr_max = 0
print("Column length", len(column))
for w_list in column:
    tmp =[]
    for str_list in w_list:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(',' ( ').replace(')',' ) ').replace('"','').split()
        item_list.extend(formatted)
        new_len = len(formatted)
        tmp.extend(formatted)
        if new_len>ingr_max:
            ingr_max=new_len
    element_list.append(tmp)

print(item_list[:10])
print(element_list[0])
print("Len Elements:", len(element_list))
ingr_vocab = set(item_list)
ingr_vocab_size = len(ingr_vocab)
print("Size:", ingr_vocab_size)
print("Max:", ingr_max)

Column length 13496
['1', '(', '3½–4-lb', '.', ')', 'whole', 'chicken', '2¾', 'tsp', '.']
['1', '(', '3½–4-lb', '.', ')', 'whole', 'chicken', '2¾', 'tsp', '.', 'kosher', 'salt', ',', 'divided', ',', 'plus', 'more', '2', 'small', 'acorn', 'squash', '(', 'about', '3', 'lb', '.', 'total', ')', '2', 'Tbsp', '.', 'finely', 'chopped', 'sage', '1', 'Tbsp', '.', 'finely', 'chopped', 'rosemary', '6', 'Tbsp', '.', 'unsalted', 'butter', ',', 'melted', ',', 'plus', '3', 'Tbsp', '.', 'room', 'temperature', '¼', 'tsp', '.', 'ground', 'allspice', 'Pinch', 'of', 'crushed', 'red', 'pepper', 'flakes', 'Freshly', 'ground', 'black', 'pepper', '⅓', 'loaf', 'good-quality', 'sturdy', 'white', 'bread', ',', 'torn', 'into', '1', 'pieces', '(', 'about', '2½', 'cups', ')', '2', 'medium', 'apples', '(', 'such', 'as', 'Gala', 'or', 'Pink', 'Lady;', 'about', '14', 'oz', '.', 'total', ')', ',', 'cored', ',', 'cut', 'into', '1', 'pieces', '2', 'Tbsp', '.', 'extra-virgin', 'olive', 'oil', '½', 'small', 'red', 'onion',

In [3]:
df["Cleaned_Ingredients"][0]

['1 (3½–4-lb.) whole chicken',
 '2¾ tsp. kosher salt, divided, plus more',
 '2 small acorn squash (about 3 lb. total)',
 '2 Tbsp. finely chopped sage',
 '1 Tbsp. finely chopped rosemary',
 '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
 '¼ tsp. ground allspice',
 'Pinch of crushed red pepper flakes',
 'Freshly ground black pepper',
 '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
 '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
 '2 Tbsp. extra-virgin olive oil',
 '½ small red onion, thinly sliced',
 '3 Tbsp. apple cider vinegar',
 '1 Tbsp. white miso',
 '¼ cup all-purpose flour',
 '2 Tbsp. unsalted butter, room temperature',
 '¼ cup dry white wine',
 '2 cups unsalted chicken broth',
 '2 tsp. white miso',
 'Kosher salt',
 'freshly ground pepper']

In [4]:
column = df["Instructions"]
# item_list = [word
#                     for w_list in column
#                     for str_list in w_list
#                     # Make period and comma separate words
#                     # # Remove parentheses and quotes
#                     for word in str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()]

item_list = []
inst_max = 0
for w_list in column:
    for str_list in w_list:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
        item_list.extend(formatted)
        new_len = len(formatted)
        if new_len>inst_max:
            inst_max=new_len
            
print(item_list[:10])
inst_vocab = set(item_list)
inst_vocab_size = len(inst_vocab)
print("Size:",inst_vocab_size)
print("Max:", inst_max)

['Pat', 'chicken', 'dry', 'with', 'paper', 'towels', 'season', 'all', 'over', 'with']
Size: 21164
Max: 247


In [5]:
column = df["Title"].to_list()
# item_list = [word
#                 for str_list in column
#     #                     # Make period and comma separate words
#     #                     # # Remove parentheses and quotes
#                 for word in str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
#                 if word]

item_list = []
title_max = 0
for str_list in column:
        # for word in str_list:
        formatted = str_list.replace('.', ' . ').replace(',',' , ').replace('(','').replace(')','').replace('"','').split()
        item_list.extend(formatted)
        new_len = len(formatted)
        if new_len>title_max:
            title_max=new_len

print(item_list[:10])
title_vocab = set(item_list)
title_vocab_size = len(title_vocab)
print("Size:", title_vocab_size)
print("Max:", title_max)

['Miso-Butter', 'Roast', 'Chicken', 'With', 'Acorn', 'Squash', 'Panzanella', 'Crispy', 'Salt', 'and']
Size: 6959
Max: 21


In [7]:
total_vocab = set(list(inst_vocab) + list(ingr_vocab) + list(title_vocab)) 
total_vocab_size = len(total_vocab)
print("Size:",total_vocab_size)
total_max = max([inst_max, ingr_max, title_max])
print("Max:",total_max)

Size: 31551
Max: 247


In [8]:
word_to_ix = {word: i for i, word in enumerate(total_vocab)}

word_to_ix_ingr = {word: i for i, word in enumerate(ingr_vocab)}
word_to_ix_inst = {word: i for i, word in enumerate(inst_vocab)}
word_to_ix_title = {word: i for i, word in enumerate(title_vocab)}

In [9]:
# from torchtext.vocab import vocab

# Get longest sentence in Ingredients or Instructions for max_len
# Use torchtext to make vocab: vocab_size = input_size

In [10]:
def pad_to_value(x, val):
    if x.shape[1]>val:
        raise ValueError(f"{x.shape[1]}>{val}")
    diff = val - x.shape[1]
    # left, right, top, bottom
    pad = (0, diff, 0, 0)
    out = torch.nn.functional.pad(x, pad, mode='constant', value=val)
    return out

In [11]:
# Test encoder:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)


# first batch
# can input list of np arrays so 0:title, 1:ingr, 2: instr
test_input = {
    "Title": 8,
    "Ingredients" : 15,
    "Instructions": 30
}

src_list = [np.random.rand(2, 8) * 10,
            np.random.rand(2, 15) * 10,
            np.random.rand(2, 30) * 10
            ]
x = [ pad_to_value(torch.tensor(t, dtype=torch.long),total_max) for t in src_list]
print("Input shape:", x[0].shape)



You are using device: cuda
Input shape: torch.Size([2, 247])


In [12]:
recipe_encoder = recipe.RecipeEncoder(device="cpu", vocab_size=total_vocab_size,
                                      max_len=total_max)
outputs = recipe_encoder.forward(x)
print(outputs)

(tensor([[[-0.4319,  1.1607,  1.2959,  ...,  1.4938,  1.5929, -0.4808],
         [ 0.5576,  0.1172,  1.6720,  ...,  1.5837,  1.1210, -1.7731],
         [ 0.3178,  0.8028,  1.5160,  ...,  0.0826, -0.3584,  1.0151],
         ...,
         [-1.3113, -0.7386, -0.7693,  ..., -0.8085, -1.4888,  1.5727],
         [-1.5756,  0.4081, -1.7119,  ..., -0.5649, -1.3221,  3.2492],
         [-1.4354,  0.6120, -0.5559,  ..., -1.2652, -0.8902,  1.1752]],

        [[-0.8781,  0.0850,  1.6982,  ...,  0.5467,  1.2825, -0.8654],
         [ 0.0965,  1.3435,  1.1319,  ...,  1.6492,  1.0559, -1.3387],
         [-0.2884,  0.2391,  2.1344,  ...,  0.1308,  0.1568,  1.1403],
         ...,
         [-1.2416, -0.7193, -0.8451,  ..., -1.0265, -1.4941,  1.8770],
         [-1.5550,  0.3511, -1.3751,  ..., -0.2427, -1.1542,  3.1476],
         [-1.0953,  0.7898, -1.0457,  ..., -1.1118, -1.5302,  1.8209]]],
       grad_fn=<CatBackward0>), tensor([[-0.1156, -0.4029, -0.6104,  ..., -0.1854,  0.4286,  0.2250],
        [-0.1